In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/emotion-detection-dataset'


/content/drive/MyDrive/emotion-detection-dataset


In [4]:
import pandas as pd

train_df = pd.read_csv('train.txt', sep=';', header=None, names=['text', 'emotion'])
test_df = pd.read_csv('test.txt', sep=';', header=None, names=['text', 'emotion'])
val_df = pd.read_csv('val.txt', sep=';', header=None, names=['text', 'emotion'])


In [ ]:
import pandas as pd

# Read the train.txt file into a pandas dataframe
train_df = pd.read_csv('/content/drive/MyDrive/emotion-detection-dataset/train.txt', sep=';', header=None, names=['text', 'emotion'])

# Display the first 5 rows of the dataframe
print(train_df.head())


                                                text  emotion
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger


In [ ]:
import pandas as pd

# Read the test.txt file into a pandas dataframe
test_df = pd.read_csv('/content/drive/MyDrive/emotion-detection-dataset/test.txt', sep=';', header=None, names=['text', 'emotion'])

# Display the first 5 rows of the dataframe
print(test_df.head())


                                                text  emotion
0  im feeling rather rotten so im not very ambiti...  sadness
1          im updating my blog because i feel shitty  sadness
2  i never make her separate from me because i do...  sadness
3  i left with my bouquet of red and yellow tulip...      joy
4    i was feeling a little vain when i did this one  sadness


In [5]:
#Final_ProjectCode_ID_1944_1863
#Naive Bayes Classifier
import pandas as pd
import numpy as np
from collections import Counter

# Read the dataset into pandas dataframes
train_df = pd.read_csv('/content/drive/MyDrive/emotion-detection-dataset/train.txt', sep=';', header=None, names=['text', 'emotion'])
test_df = pd.read_csv('/content/drive/MyDrive/emotion-detection-dataset/test.txt', sep=';', header=None, names=['text', 'emotion'])
val_df = pd.read_csv('/content/drive/MyDrive/emotion-detection-dataset/val.txt', sep=';', header=None, names=['text', 'emotion'])

# Create a dictionary to store the word frequencies for each emotion
freqs = {}

# Iterate through the training set and count the word frequencies for each emotion
for i in range(len(train_df)):
    words = train_df.iloc[i]['text'].split()
    emotion = train_df.iloc[i]['emotion']
    if emotion not in freqs:
        freqs[emotion] = Counter()
    freqs[emotion].update(words)

# Compute the total number of words in each emotion
n_words = {}
for emotion in freqs.keys():
    n_words[emotion] = sum(freqs[emotion].values())

# Compute the vocabulary size
vocab = set()
for emotion in freqs.keys():
    vocab.update(freqs[emotion].keys())
vocab_size = len(vocab)

# Define a function to compute the log probabilities for each word in each emotion
def log_probs(text, freqs, n_words, vocab_size, alpha=1):
    log_prob = {}
    for emotion in freqs.keys():
        log_prob[emotion] = np.log(n_words[emotion] + alpha * vocab_size)
        for word in text.split():
            log_prob[emotion] += np.log(freqs[emotion][word] + alpha)
            log_prob[emotion] -= np.log(n_words[emotion] + alpha * vocab_size)
    return log_prob

# Define a function to predict the emotion of a given text using Naive Bayes Classifier
def predict(text, freqs, n_words, vocab_size, alpha=1):
    log_prob = log_probs(text, freqs, n_words, vocab_size, alpha)
    emotion = max(log_prob, key=log_prob.get)
    return emotion

# Test the model on a user input
text = input("Enter a sentence: ")
emotion = predict(text, freqs, n_words, vocab_size)
print("Naive Bayes Classifier Prediction: ", emotion)

# Compute the accuracy of the model on the test set
y_true = test_df['emotion'].tolist()
y_pred = [predict(text, freqs, n_words, vocab_size) for text in test_df['text'].tolist()]
accuracy = sum(np.array(y_true) == np.array(y_pred)) / len(y_true)
print("Naive Bayes Classifier Accuracy: ", accuracy)


Enter a sentence: I am feeling happy today
Naive Bayes Classifier Prediction:  joy
Naive Bayes Classifier Accuracy:  0.773


In [ ]:
#Final_ProjectCode_ID_1944_1863
#K-Nearest Neighbors
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Read in the data using pandas
train_df = pd.read_csv('train.txt', sep=';', header=None, names=['text', 'emotion'])
test_df = pd.read_csv('test.txt', sep=';', header=None, names=['text', 'emotion'])
val_df = pd.read_csv('val.txt', sep=';', header=None, names=['text', 'emotion'])

# Combine the data into one dataframe
data_df = pd.concat([train_df, test_df, val_df])

# Vectorize the text using a bag-of-words model
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_df['text'].values)

# Create the target labels
y = data_df['emotion'].values

# Split the data into training and testing sets
X_train = X[:len(train_df)]
y_train = y[:len(train_df)]
X_test = X[len(train_df):(len(train_df) + len(test_df))]
y_test = y[len(train_df):(len(train_df) + len(test_df))]

# Define the KNN model and fit the training data
k = 5  # Number of neighbors to consider
model = KNeighborsClassifier(n_neighbors=k)
model.fit(X_train, y_train)

# Make predictions for user input
user_input = input("Enter a sentence: ")
user_input_vectorized = vectorizer.transform([user_input])
prediction = model.predict(user_input_vectorized)[0]

print("K-Nearest Neighbors Classifier Prediction:", prediction)

# Calculate the accuracy of the model using cross-validation
scores = cross_val_score(model, X, y, cv=5)
accuracy = np.mean(scores)

print("K-Nearest Neighbors Classifier Accuracy:", accuracy)


Enter a sentence: i feel beautifully emotional knowing that these women of whom i knew just a handful were holding me and my baba on our journey
K-Nearest Neighbors Classifier Prediction: sadness
K-Nearest Neighbors Classifier Accuracy: 0.4014500000000001


In [ ]:
#Final_ProjectCode_ID_1944_1863
#Linear Regression
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Define the Linear Regression Classifier
class LinearRegression:
    def __init__(self, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        # Initialize the weights and bias to zero
        n_samples = X.shape[0]
        self.weights = np.zeros(X.shape[1])
        self.bias = 0

        # Gradient descent
        for _ in range(self.n_iters):
            y_predicted = np.dot(X, self.weights) + self.bias
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        y_predicted = np.dot(X, self.weights) + self.bias
        return y_predicted


# Load the dataset
train_df = pd.read_csv('train.txt', sep=';', header=None, names=['text', 'emotion'])
test_df = pd.read_csv('test.txt', sep=';', header=None, names=['text', 'emotion'])
val_df = pd.read_csv('val.txt', sep=';', header=None, names=['text', 'emotion'])

# Preprocess the dataset
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
X_train = vectorizer.fit_transform(train_df['text']).toarray()
le = LabelEncoder()
y_train = le.fit_transform(train_df['emotion'])

# Train the Linear Regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Make predictions on the test set
X_test = vectorizer.transform(test_df['text']).toarray()
y_test = le.transform(test_df['emotion'])
y_pred = lr.predict(X_test)

# Convert the predicted numerical labels back to categorical labels
y_pred = le.inverse_transform(y_pred.astype(int))

# Calculate the accuracy of the Linear Regression model
accuracy = accuracy_score(test_df['emotion'], y_pred)
print("Linear Regression Classifier Accuracy:", accuracy)

# Test the Linear Regression model with user input
user_input = input("Enter a sentence: ")
X_input = vectorizer.transform([user_input]).toarray()
y_input = le.inverse_transform(lr.predict(X_input).astype(int))
print("Linear Regression Classifier Prediction:", y_input[0])


Linear Regression Classifier Accuracy: 0.3475
Enter a sentence: I am feeling happy today
Linear Regression Classifier Prediction: joy
